In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.table import Table
from astropy import table
from astroquery.gaia import Gaia
from scipy.optimize import minimize
from scipy.stats import scoreatpercentile
from scipy.interpolate import RBFInterpolator, LinearNDInterpolator
import stam
from tqdm import tqdm
import WD_models
from os import path
import extinction
%matplotlib tk

# Sources

In [3]:
# clusters = Table.read('../data/hunt_clusters/clusters.csv')
# members = Table.read('../data/hunt_clusters/members.csv')
sources = Table.read('../table_C.fits', format='fits')

In [14]:
# from astroquery.vizier import Vizier
# clstrs = Vizier(catalog='J/MNRAS/504/356',columns=['Cluster','N','logage','e_logage','[Fe/H]','e_[Fe/H]','Av','e_Av','FileName'],row_limit=-1).query_constraints()[0]

# clstrs.rename_columns(clstrs.colnames,['Cluster','N','logage','e_logage','Fe/H','e_Fe/H','Av','e_Av','FileName'])
# # sources = Table.read('../table_extra.fits')
# sources['idx'] = np.full(len(sources),'---',dtype=object)

# cut = (sources['nss_solution_type'] == 'Orbital') | (sources['nss_solution_type'] == 'AstroSpectroSB1')
# sources = sources[cut]

# nms = np.unique(sources['cluster'])
# for n in nms:
#     for i, s in enumerate(sources[sources['cluster'] == n]):
#         j = np.where(sources['source_id'] == s['source_id'])[0][0]
#         sources[j]['idx'] = n+'_'+str(i)

# sources['idx'] = sources['idx'].astype(str)
# sources['age'] = np.full(len(sources),np.nan)
# sources['[Fe/H]'] = np.full(len(sources),np.nan)
# sources['Av'] = np.full(len(sources),np.nan)

# for j in range(len(sources)):
#     if sources[j]['cluster'] == 'Hyades':
#         sources[j]['age'] = 680
#         sources[j]['[Fe/H]'] = 0.14
#         sources[j]['Av'] = 0.01 * 3.1
#     if sources[j]['cluster'] == 'IC_2602':
#         sources[j]['age'] = 60
#         sources[j]['[Fe/H]'] = -0.02
#         sources[j]['Av'] = 0.05 * 3.1
#     if sources[j]['cluster'] == 'NGC_2287':
#         sources[j]['age'] = 200
#         sources[j]['[Fe/H]'] = -0.11
#         sources[j]['Av'] = 0.03 * 3.1
#     if sources[j]['cluster'] == 'NGC_2547':
#         sources[j]['mh'] = 60
#         sources[j]['[Fe/H]'] = 0.01
#         sources[j]['Av'] = 0.06 * 3.1

# for j in range(len(sources)):
#     clstr = sources[j]['cluster']
#     if clstr in clstrs['cluster']:
#         i = np.where(clstrs['cluster'] == clstr)[0][0]
#         sources[j]['age'] = clstrs[i]['age']
#         sources[j]['[Fe/H]'] = clstrs[i]['Fe_H']
#         sources[j]['Av'] = clstrs[i]['ebv'] * 3.1
    
# sources.sort('idx')

# PARSEC functions

In [4]:
# PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
PARSEC_path = '../OCFit/gaiaDR2/grids/'
models = stam.getmodels.read_parsec(path=PARSEC_path)


def choose_model(mh):
    if (-0.6 <= mh) and (mh <= 0.05):
        PARSEC_path = '../data/PARSECv2.0/w_i=0.6/'
        models = stam.getmodels.read_parsec(path=PARSEC_path)
    else:
        PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
        models = stam.getmodels.read_parsec(path=PARSEC_path)
    return PARSEC_path,models

def get_tracks(mh,age):
    
    color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G" ## no rotation    
    age_res = np.min(abs(10**np.unique(models['logAge'].data) * 1e-9 - age * 1e-3)) * 1.05 + 1e-3
    mh_res = np.min(abs(np.unique(models['MH'].data) - mh)) + 0.005
   
    stage_min = 0  # pre-main sequence
    stage_max = 5  # red giant branch
    mass_min = 0  # [Msun]
    mass_max = 8  # [Msun]
    tracks = stam.gentracks.get_isotrack(models, [age * 1e-3, mh], params=("age", "mh"), return_table=True,
                                    age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = mh_res,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)
    return tracks

def get_track_idx(mh,age):
    color_fil_1, color_fil_2, mag_fil = "G_BP", "G_RP", "G" ## no rotation    
    age_res = np.min(abs(10**np.unique(models['logAge'].data) * 1e-9 - age * 1e-3)) * 1.05
    stage_min = 0  # pre-main sequence
    stage_max = 5  # red giant branch
    mass_min = 0  # [Msun]
    mass_max = 8  # [Msun]
    track_idx = stam.gentracks.get_isotrack(models, [age * 1e-3, mh], params=("age", "mh"), return_idx=True,
                                    age_res=age_res, mass_min=mass_min, mass_max=mass_max, mh_res = 0.025,
                                    stage=None, stage_min=stage_min, stage_max=stage_max, sort_by="age", color_filter1=color_fil_1, color_filter2=color_fil_2,
                mag_filter=mag_fil)   
    return track_idx

def get_age_mh_grid():
    age = 10**np.unique(models['logAge'].data) * 1e-6
    mh = np.unique(models['MH'].data)
    age,mh = np.meshgrid(age,mh)
    return age,mh


# c = (models['label'] <= 5) & (models['Mini']<=8)
# p1 = models[c]['Mass']
# p2 = models[c]['logAge']
# p3 = models[c]['MH']
# pdep1 = models[c]['G_BPmag'] - models[c]['G_RPmag']
# pdep2 = models[c]['Gmag']

# mass_to_mag = LinearNDInterpolator(np.vstack(list(zip(p1,p2,p3))), pdep2)
# mass_to_color = LinearNDInterpolator(np.vstack(list(zip(p1,p2,p3))), pdep1)

# def get_isochrone(mh,age):
#     mass_arr = get_tracks(mh,age)['mass']
#     gmag = mass_to_mag(mass_arr,np.log10(age*1e6),mh)
#     bprp = mass_to_color(mass_arr,np.log10(age*1e6),mh)
#     return Table([mass_arr,bprp,gmag],names=['mass','bp_rp','mg'])

# _ = get_isochrone(0,101)

# Cluster parameters and primary mass

In [4]:
## mass interpolations, cluster cmd plots

def tracks2grid(tracks, xparam = "bp_rp", yparam = "mg", xstep=0.05, ystep=0.05):
    # auxiliary to interp_mass_realization
    xmin = np.min(np.around(tracks[xparam], -int(np.round(np.log10(xstep)))))
    xmax = np.max(np.around(tracks[xparam], -int(np.round(np.log10(xstep)))))
    ymin = np.min(np.around(tracks[yparam], -int(np.round(np.log10(ystep)))))
    ymax = np.max(np.around(tracks[yparam], -int(np.round(np.log10(ystep)))))
    x, y = np.meshgrid(np.arange(xmin, xmax, xstep), np.arange(ymin, ymax, ystep))
            
    return x, y, xmin, xmax, ymin, ymax

def interp_mass_realization(age,mh,av,bp_rp,mg):
    # For a single realization of cluster and source parameters, interpolate the mass

    # get the isotrack for this realization of the cluster parameters
    tracks = get_tracks(mh,age)
    x = np.array(tracks["bp_rp"])
    y = np.array(tracks["mg"])
    z = np.array(tracks["mass"])
    # Add small noise to the isochrone (to avoid singularity problems)
    x = x + np.random.normal(0, 0.0001, len(x))
    y = y + np.random.normal(0, 0.0001, len(y))
    z = z + np.random.normal(0, 0.0001, len(z))
    # apply extinction to the isochrone
    ag, e_bprp = extinction.get_AG_EBPRP(av,x)
    y = y + ag
    x = x + e_bprp
    # interpolate this instance of color and magnitude along the isochrone
    xstep, ystep = 0.05, 0.05
    grid_x, grid_y, xmin, xmax, ymin, ymax = tracks2grid(tracks, xstep=xstep, ystep=ystep)    
    fun_type = "linear"
    interp = RBFInterpolator(np.array([x, y]).T, z, kernel=fun_type)
    grid = np.array([grid_x, grid_y])
    grid_flat = grid.reshape(2, -1).T
    grid_z = interp(grid_flat).reshape(grid_x.shape)
    mass = interp(np.array([[bp_rp, mg]]))[0]
    return mass

def photometric_mass(idx,save=False,plot=False,n_realizations=100):        
    # Run monte carlo simulation to estimate primary pass and error
    j = np.where(sources['idx'] == idx)[0][0]
    age, e_age = sources[j]['age'], sources[j]['e_age']
    mh, e_mh = sources[j]['[Fe/H]'], sources[j]['e_[Fe/H]']
    av, e_av = sources[j]['Av'], sources[j]['e_Av']
    mg = sources[j]['mg']
    bp_rp = sources[j]['bp_rp']

    # estimate photometric errors
    e_g= 2.5*np.log(10)*sources[j]['phot_g_mean_flux_error']/sources[j]['phot_g_mean_flux']
    e_mg = np.sqrt(e_g**2 + (2.17 / sources[j]['parallax_over_error'])**2)
    e_bp = 2.5 * np.log(10) * sources[j]['phot_bp_mean_flux_error'] / sources[j]['phot_bp_mean_flux']
    e_rp = 2.5 * np.log(10) * sources[j]['phot_rp_mean_flux_error'] / sources[j]['phot_rp_mean_flux']
    e_bp_rp = np.sqrt(e_bp**2 + e_rp**2)

    # run monte carlo simulation
    age_vec = np.random.normal(age,e_age,n_realizations)
    mh_vec = np.random.normal(mh,e_mh,n_realizations)
    av_vec = np.random.normal(av,e_av,n_realizations)
    bp_rp_vec = np.random.normal(bp_rp,e_bp_rp,n_realizations)
    mg_vec = np.random.normal(mg,e_mg,n_realizations)
    mass_vec = np.zeros(n_realizations)
    for i in range(n_realizations):
        try:
            mass_vec[i] = interp_mass_realization(age_vec[i],mh_vec[i],av_vec[i],bp_rp_vec[i],mg_vec[i])
        except:
            mass_vec[i] = np.nan
    m1 = np.nanmean(mass_vec)
    m1_err = np.nanstd(mass_vec)
    plot_mass_interp(age,mh,av,bp_rp,mg,m1,m1_err,idx,sources[j]['cluster'],save,plot)
    return m1,m1_err

def get_cluster_members(cluster):
    # read the cluster member data from our premade files, apply quality cuts

    filepath = path.join('..','OCFit','gaiaDR2','data',cluster+'.dat')
    obs = np.genfromtxt(filepath,names=True,delimiter=',',dtype=None)
    if path.exists(filepath.replace('.dat','_D.dat')):
        obsD = np.genfromtxt(filepath.replace('.dat','_D.dat'),names=True,delimiter=',',dtype=None)
        obs = np.concatenate((obs,obsD))

    mg = obs['Gmag'] - 5 * np.log10(1000/obs['Plx']) + 5
    #remove nans para fazer os plots
    cond1 = np.isfinite(obs['Gmag'])
    cond2 = np.isfinite(obs['BPmag'])
    cond3 = np.isfinite(obs['RPmag'])
    
    cond4 = obs['RFG'] > 50.0
    cond5 = obs['RFBP'] > 20.0
    cond6 = obs['RFRP'] > 20.0
    cond7 = obs['E_BR_RP_'] < 1.3+0.06*(obs['BPRP'])**2
    cond8 = obs['E_BR_RP_'] > 1.0+0.015*(obs['BPRP'])**2
    cond9 = obs['Nper'] > 8
    cond10 = obs['fidelity_v2'] > 0.5
    cond11 = (mg < 9) | (obs['BPRP'] > 0)       
    ind  = np.where(cond1&cond2&cond3&cond4&cond5&cond6&cond7&cond8&cond9&cond10&cond11)
    obs = obs[ind]
    obs = Table(obs)
    obs = table.unique(obs,keys='source_id')
    return obs
    
def plot_cmd(idx,cluster,age,mh,ebv,save,plot):
        memb = get_cluster_members(cluster)
        memb['mg'] = memb['Gmag'] - 5 * np.log10(1000/memb['Plx']) + 5

        mg = memb['mg']
        bp_rp = memb['BPRP']
        mg0 = sources[sources['idx']== idx]['mg']
        bp_rp0= sources[sources['idx']== idx]['bp_rp']

        track = get_tracks(mh,age)
        bp_rp_model = np.array(track['bp_rp'])
        mg_model = np.array(track['mg'])
        ag, e_bprp = extinction.get_AG_EBPRP(3.1*ebv,bp_rp_model)
        mg_model = mg_model + ag
        bp_rp_model = bp_rp_model + e_bprp

        fig,ax = plt.subplots(dpi=300)
        ax.scatter(bp_rp,mg,s=5,c='r',label='Cluster members',zorder=5)
        ax.scatter(bp_rp0,mg0,s=25,c='g',label='Candidate',zorder = 10)
        ax.plot(bp_rp_model,mg_model, 'ko', markersize=1,label=f'PARSEC age={int(age)} Myr, [Fe/H]={mh:.2f}, E(B-V)={ebv:.3f}')
        ax.set_xlabel('BP-RP')
        ax.set_ylabel('G')
        ax.invert_yaxis()
        ax.legend(loc='lower left',frameon=False)
        ax.set_title('Candidate '+ str(idx) + ' in cluster ' + cluster)
        if save:
            fig.savefig(f'../img/cmd/{idx}_'+cluster+'.png')
        if not plot:
            plt.close()

def plot_mass_interp(age,mh,av,bp_rp,mg,m1,m1_err,idx,clstr,save,plot):
    # For the final mass estimate, plot the mass interpolation
    tracks = get_tracks(mh,age)
    x = np.array(tracks["bp_rp"])
    y = np.array(tracks["mg"])
    z = np.array(tracks["mass"])
    # Add small noise to the isochrone (to avoid singularity problems)
    x = x + np.random.normal(0, 0.0001, len(x))
    y = y + np.random.normal(0, 0.0001, len(y))
    z = z + np.random.normal(0, 0.0001, len(z))
    # apply extinction to the isochrone
    ag, e_bprp = extinction.get_AG_EBPRP(av,x)
    y = y + ag
    x = x + e_bprp
    # interpolate this instance of color and magnitude along the isochrone
    xstep, ystep = 0.05, 0.05
    grid_x, grid_y, xmin, xmax, ymin, ymax = tracks2grid(tracks, xstep=xstep, ystep=ystep)    
    fun_type = "linear"
    interp = RBFInterpolator(np.array([x, y]).T, z, kernel=fun_type)
    grid = np.array([grid_x, grid_y])
    grid_flat = grid.reshape(2, -1).T
    grid_z = interp(grid_flat).reshape(grid_x.shape)
    # plot
    fig, ax = plt.subplots(dpi=300, tight_layout=True)
    ax.plot(x, y, 'ko', markersize=1,label=f'PARSEC age={int(age)} Myr, [Fe/H]={mh:.2f}, E(B-V)={av/3.1:.3f}')
    h = ax.imshow(grid_z, origin="lower", extent=[xmin, xmax, ymin, ymax], cmap='Oranges', aspect='auto')
    ax.scatter(bp_rp,mg,s=25,c='g',label=f'Candidate $M_1$={m1:.2f} $\pm$ {m1_err:.2f} $M_\odot$',zorder = 10)
    plt.colorbar(h, label=r"Mass (M$_\odot$)")
    ax.set_xlabel(r"$G_\text{BP}-G_\text{RP}$")
    ax.set_ylabel(r"$G$")
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(ymin, ymax)
    ax.set_title(f'Candidate {idx} in cluster {clstr}')
    ax.invert_yaxis()
    ax.legend(loc='lower left',frameon=False,fontsize=8)
    
    if save:
        fig.savefig(f'../img/mass_interp/{idx}_'+clstr+'.png')
    if not plot:
        plt.close()

In [ ]:
plot = False
save = True
sources = Table.read('../table_B.fits')
new_sources = sources.copy()
m1_col = []
m1_err_col = []

# fitting all with parsec v1.2S
# PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
PARSEC_path = '../OCFit/gaiaDR2/grids/'
models = stam.getmodels.read_parsec(path=PARSEC_path)
for idx in new_sources['idx']:
    age = new_sources[new_sources['idx'] == idx]['age'][0]
    mh = new_sources[new_sources['idx'] == idx]['[Fe/H]'][0]
    ebv = new_sources[new_sources['idx'] == idx]['Av'][0] / 3.1
    cluster = new_sources[new_sources['idx'] == idx]['cluster'][0]
    if np.isnan(age):
        # if unable to fit age, skip the candidate
        # new_sources.remove_row(np.where(new_sources['idx'] == idx)[0][0])
        m1_col.append(np.nan)
        m1_err_col.append(np.nan)
    else:
        try:
            print(f'Photometric mass for candidate {idx}...')
            m1, m1_err = photometric_mass(idx,save=True,plot=False,n_realizations=100)
            m1_col.append(m1)
            m1_err_col.append(m1_err)
        except:
            print(f'Error in photometric mass for candidate {idx}')
            m1_col.append(np.nan)
            m1_err_col.append(np.nan)
        plot_cmd(idx,cluster,age,mh,ebv,save,plot)

In [24]:
new_sources['m1'] = m1_col
new_sources['m1_err'] = m1_err_col

# new_sources.write('../table_B.fits',overwrite=True)
sources = new_sources

# AMRF

In [5]:
# Auxiliary functions, gaia passbands

# ------------ AMRF limits ----------------
from astropy.io import ascii
from synphot import SourceSpectrum, ReddeningLaw
from synphot.models import BlackBodyNorm1D
from synphot.units import convert_flux
from astropy import constants as const
from astropy import units as u
import json 
from uncertainties import unumpy as unp, ufloat
from uncertainties import correlated_values_norm, correlation_matrix
import warnings

def blackbody(temperature, wavelength, ebv=None, extinction_model='mwavg'):
    bb = SourceSpectrum(BlackBodyNorm1D, temperature=temperature*u.K)  # [photons s^-1 cm^-2 A^-1]
    # if ebv is not None:
    #     # apply extinction
    #     ext = ReddeningLaw.from_extinction_model(extinction_model).extinction_curve(ebv)
    #     bb = bb * ext
    bb = bb(wavelength)/(const.R_sun / const.kpc) ** 2  # undo synphot normalization (but leave the pi factor from integration over half a sphere)
    bb = convert_flux(wavelength, bb, 'flam')  # [flam] = [erg s^-1 cm^-2 A^-1]
    bb = bb.to(u.erg/u.s/u.cm**2/u.angstrom)  # express in normal astropy units
    return bb


def mlogg2radius(m, logg):
    g = 10**logg*u.cm/u.s**2
    r = np.sqrt(const.G*m/g)
    return r.to(u.Rsun).value


def calc_synth_phot(wavelength, flux, bandpass):
    dlambda = np.diff(wavelength)
    dlambda = np.concatenate([dlambda, np.array([dlambda[-1]])])

    # assuming a photon-counting device
    phot = np.sum(dlambda*bandpass*wavelength*flux)/np.sum(dlambda*bandpass*wavelength)
    
    return phot


amrf = lambda q, S : q/(1+q)**(2/3)*(1 - S*(1+q)/(q*(1+S)))

gaia_passband = ascii.read('../data/other/passband.dat', names=["wl", "gPb", "gPbError", "bpPb", "bpPbError", "rpPb", "rpPbError"])

# replace missing values with NaNs
for col in gaia_passband.itercols():
    col[col == 99.99] = 0
    
gaia_passband['wl'] *= 10  # [A]

# ---------- AMRF -------------------

limiting_curves = Table.read('../data/other/AMRF_limiting_curves.fits')
# Retrieve the conservative limiting AMRF values for some primary mass
# --------------
def Atr(m1):
    j = np.argmin(np.abs(m1 - limiting_curves['m1'].data))
    return limiting_curves['Atr'][j]
def Ams(m1):
    j = np.argmin(np.abs(m1 - limiting_curves['m1'].data))
    return limiting_curves['Ams'][j]

# =============================================================================
#                Auxil routines to obtain the covariance matrix
# =============================================================================

# 1) get the list of parameters from the solution type
def get_par_list(solution_type=None):
    if (solution_type is None) or (solution_type=='Orbital'):
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'eccentricity', 'period', 't_periastron')

    elif (solution_type=='OrbitalAlternative') or (solution_type=='OrbitalAlternativeValidated') \
            or (solution_type=='OrbitalTargetedSearch') or (solution_type=='OrbitalTargetedSearchValidated'):
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'period', 'eccentricity', 't_periastron')

    elif solution_type=='AstroSpectroSB1':
        return ('ra', 'dec', 'parallax', 'pmra', 'pmdec', 'a_thiele_innes',
                'b_thiele_innes', 'f_thiele_innes', 'g_thiele_innes',
                'c_thiele_innes', 'h_thiele_innes', 'center_of_mass_velocity',
                'eccentricity', 'period', 't_periastron')


# 2) Get the order of parameters in the covariance matrix, for a given bit index.
def bit_index_map(bit_index):
    if bit_index==8191:
        return ['ra','dec','parallax','pmra','pmdec','A','B','F','G', 'e','P', 'T']
    elif bit_index==8179:
        return ['ra','dec','parallax','pmra','pmdec','A','B','F','P', 'T']
    elif bit_index==65535:
        return ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'A', 'B', 'F', 'G', 'C', 'H', 'gamma','e', 'P', 'T']
    elif bit_index==65435:
        return ['ra', 'dec', 'parallax', 'pmra', 'pmdec', 'A', 'B', 'F', 'H', 'gamma', 'P', 'T']
    else:
        return None
    

# 3) Generate the correlation matrix
def make_corr_matrix(input_table, pars=None):
    """
    INPUT:
    input_table nss_two_body_orbit table.
    pars : list
            list of parameters for the corresponding solution of the desired
              target, in the same order as they appear in the Gaia table.
      """
    if pars is None:
        pars = get_par_list()

    # read the correlation vector
    s1 = input_table['corr_vec'].replace('\n','')   
    s1 = s1.replace(' ',',')
    s1 = s1.replace('--','0')
    corr_vec = list(json.loads(s1))
    # set the number of parameters in the table
    n_pars = len(pars)
    # define the correlation matrix.
    corr_mat = np.ones([n_pars, n_pars], dtype=float)

    # Read the matrix (lower triangle)
    ind = 0
    for i in range(n_pars):
        for j in range(i):
            corr_mat[j][i] = corr_vec[ind]
            corr_mat[i][j] = corr_vec[ind]
            ind += 1

    return corr_mat


# 4) Get the NSS data 
def get_nss_data(input_table, source_id):

    target_idx = np.argwhere(input_table['source_id'] == source_id)[0][0]
    pars = get_par_list(input_table['nss_solution_type'][target_idx])
    corr_mat = make_corr_matrix(input_table[target_idx], pars=pars)

    mu, std = np.zeros(len(pars)), np.zeros(len(pars))
    for i, par in enumerate(pars):
        try:
            mu[i] = input_table[par][target_idx]
            std[i] = input_table[par + '_error'][target_idx]
        except KeyError:
            mu[i], std[i] = np.nan, np.nan

    nan_idxs = np.argwhere(np.isnan(corr_mat))
    corr_mat[nan_idxs[:, 0], nan_idxs[:, 1]] = 0.0

    return mu, std, corr_mat

def multivar_sample(mu, sigma, corr, n):
    cov = corr*(sigma[:, None] * sigma[None, :])
    # l = spla.cholesky(cov)
    # z = np.random.normal(size=(n, mu.shape[0]))
    # return z.dot(l) + mu
    return np.random.multivariate_normal(mu, cov, size=n)

# =============================================================================
#                      calc parameters.
# =============================================================================
# Here we calculate the AMRF, qmin, etc, assuming that the mass of the luminous star 
# is exactly one solar mass. This is just for the red-clump stars...
def calc_AMRF(par_in, par_in_errors, corr_matrix, m1, bit_index=8191):
    """
    For the given set of orbital parameters by Gaia, this function calculates
    the standard geometrical elements (a, omega, Omega, and i). If the error
    estimates and covariance matrix are prodived, the error estimates on the
    calculated parameters are returned as well.

    Input: thiele_innes: Thiele Innes parameters [A,B,F,G] in milli-arcsec
           thiele_innes_errors : Corresponding errors.
           corr_matrix : Corresponding  4X4 correlation matrix.

  Output: class-III probability via monte carlo
    """
    # Read the coefficients and assign the correlation matrix.
    # Create correlated quantities. If the error is nan we assume 1e-6...
    par_in_errors[np.isnan(par_in_errors)] = 1e-6
    par_list = correlated_values_norm([(par_in[i], par_in_errors[i]) for i in np.arange(len(par_in))], corr_matrix)
    key_list = bit_index_map(bit_index)

    par = {key_list[i]: par_list[i] for i in np.arange(len(key_list))}
    par['mass'] = m1

    # Add the G Thiele-Innes parameter if needed.
    if (bit_index == 8179) | (bit_index == 65435):
        G = -par['A']*par['F']/par['B']
    else:
        G = par['G']

    # This in an intermediate step in the formulae...
    p = (par['A'] ** 2 + par['B'] ** 2 + G ** 2 + par['F'] ** 2) / 2.
    q = par['A'] * G - par['B'] * par['F']

    # Calculate the angular semimajor axis (already in mas)
    a_mas = unp.sqrt(p + unp.sqrt(p ** 2 - q ** 2))

    # Calculate the inclination and convert from radians to degrees
    i_deg = unp.arccos(q / (a_mas ** 2.)) * (180 / np.pi)
    
    try:
        if par.get("d") is not None:
            K_kms = 4.74372*unp.sqrt(par['C'] ** 2 + par['H'] ** 2)*(2*np.pi)/(par['P']/ 365.25)/np.sqrt(1-par['e']**2)
            acc   = 2*K_kms/par['P']
        else:
            K_kms = 4.74372*unp.sqrt(par['C'] ** 2 + par['H'] ** 2)*(2*np.pi)/(par['P']/ 365.25)
            acc   = K_kms/par['P']/4
    except:
        K_kms = ufloat(999, 999) 
        acc   = ufloat(999,999)

    # Calculate the AMRF
    try:
        AMRF = a_mas / par['parallax'] * par['mass'] ** (-1 / 3)  * (par['P']/ 365.25) ** (-2 / 3)

        # Calculate AMRF q
        y = AMRF ** 3
        h = (y/2 + (y**2)/3 + (y**3)/27
             + np.sqrt(3)/18*y*unp.sqrt(4*y+27))**(1/3)
        q = h + (2*y/3 + (y**2)/9)/h + y/3
    except:
        AMRF = ufloat(np.nan, np.inf) 
        q    = ufloat(np.nan, np.inf) 
        
    # Extract expectancy values and standard deviations
    pars = np.array([unp.nominal_values(AMRF),
                         unp.nominal_values(q),
                         unp.nominal_values(a_mas),
                         unp.nominal_values(i_deg),
                         unp.nominal_values(K_kms),
                         unp.nominal_values(acc)])

    pars_error = np.array([unp.std_devs(AMRF),
                               unp.std_devs(q),
                               unp.std_devs(a_mas),
                               unp.std_devs(i_deg),
                               unp.std_devs(K_kms),
                               unp.std_devs(acc)])

    return pars, pars_error

def class_probs(Atr,Ams,par_in, par_in_errors,
                  m1, m1_error, corr_matrix, bit_index=8191, n=1e2, factor=1.0):
    """
    For the given set of orbital parameters by Gaia, this function calculates
    the standard geometrical elements (a, omega, Omega, and i). If the error
    estimates and covariance matrix are prodived, the error estimates on the
    calculated parameters are returned as well.

    Input: thiele_innes: Thiele Innes parameters [A,B,F,G] in milli-arcsec
           thiele_innes_errors : Corresponding errors.
           corr_matrix : Corresponding  4X4 correlation matrix.

  Output: physical and geometrical parameters
    """
    r_3 = 0
    r_2 = 0
    par_in_errors[np.isnan(par_in_errors)] = 1e-6
    vecs = multivar_sample(par_in, par_in_errors, corr_matrix, int(n))
    key_list = bit_index_map(bit_index)

    for vec in vecs:
        par = {key_list[i]: vec[i] for i in np.arange(len(key_list))}
        par['mass'] = m1_error*np.random.randn() + m1

        # Add the G Thiele-Innes parameter if needed.
        if (bit_index == 8179) | (bit_index == 65435):
            par['G'] = -par['A'] * par['F'] / par['B']

        # This in an intermediate step in the formulae...
        p = (par['A'] ** 2 + par['B'] ** 2 + par['G'] ** 2 + par['F'] ** 2) / 2.
        q = par['A'] * par['G'] - par['B'] * par['F']

        # Calculate the semimajor axis (already in mas)
        a_mas = np.sqrt(p + np.sqrt(p ** 2 - q ** 2))

        # Calculate the AMRF
        AMRF = a_mas / par['parallax'] * par['mass'] ** (-1 / 3) * (par['P']/ 365.25) ** (-2 / 3)

        try:
            if 0 < par['e'] < 1:
                if AMRF > Atr * factor:
                    r_3 += 1
                elif Ams * factor < AMRF < Atr * factor:
                    r_2 += 1
        except KeyError:
            pass

    return (n-r_2-r_3)/n, r_2/n, r_3/n #(no_detections + detections)

# =============================================================================
#                       Read the data from the NSS table
# =============================================================================
def add_astrometric_parameters(data):
    # Here we only calculate (but don't assign class 3 probabilities!
    # We get the data table, arrange the arrays, calculate the astrometric
    # coefficients and plug it all back into the table.

    # Initialize the arrays
    # ---------------------
    # We need to calculate the AMRF, mass ratio, angular semi-major axis, orbtial inclination
    # and order-of-magnitude acceleration. We also want their uncertainties.
    count_good, count_bad = 0, 0
    A, q, a_mas, i_deg, K_kms, acc, P1, P2, P3 = np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                              np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                              np.full(len(data), np.nan), np.full(len(data), np.nan), \
                              np.full(len(data), np.nan), np.full(len(data), np.nan), np.full(len(data), np.nan) 

    Ae, qe, a_mase, i_dege, K_kmse, acce = np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                                   np.full(len(data), np.nan),  np.full(len(data), np.nan), \
                                   np.full(len(data), np.nan),  np.full(len(data), np.nan)

    # Now go one by one and calculate the AMRF
    # ----------------------------------------
    for idx in tqdm(range(len(data['source_id']))):
        if data[idx]['nss_solution_type'] not in ['Orbital','AstroSpectroSB1']:
            continue
        # Read the NSS solutin values.
        sid = data['source_id'][idx]
        mu, std, corr_mat = get_nss_data(data, sid)
        m1 = data['m1'][idx]
        m1_error = data['m1_err'][idx]
        if np.ma.is_masked(m1):
            print(idx)
            pass
        Ams_idx = data['Ams'][idx]
        Atr_idx = data['Atr'][idx]
        if np.ma.is_masked(Ams_idx) | np.ma.is_masked(Atr_idx):
            Ams_idx = Ams(m1)
            Atr_idx = Atr(m1)
            
        vals, stds = calc_AMRF(mu, std, corr_mat, m1, bit_index=data['bit_index'][idx])
        p1, p2, p3 = class_probs(data['Atr'][idx],data['Ams'][idx],mu, std, m1, m1_error, corr_mat, bit_index=data['bit_index'][idx], n = 1e4)
        try:
            A[idx], Ae[idx]  = vals[0], stds[0]
            q[idx], qe[idx]  = vals[1], stds[1]
            a_mas[idx], a_mase[idx]  = vals[2], stds[2]
            i_deg[idx], i_dege[idx]  = vals[3], stds[3]
            K_kms[idx], K_kmse[idx]  = vals[4], stds[4]
            acc[idx],   acce[idx]    = vals[5], stds[5]
            P1[idx], P2[idx], P3[idx] = p1, p2, p3
        except:
            pass
    
    # Store it all back in the original data structure.
    data['AMRF'], data['AMRF_error'] = A, Ae
    data['AMRF_q'], data['AMRF_q_error'] = q, qe
    data['a_mas'], data['a_mas_error'] = a_mas, a_mase
    data['i_deg'], data['i_deg_error'] = i_deg, i_dege
    data['K_kms'], data['K_kms_error'] = K_kms, K_kmse
    data['acc_kmsd'], data['acc_kmsd_error'] = acc, acce
    data['classI_prob'] = P1
    data['classII_prob'] = P2
    data['classIII_prob'] = P3
    return data


In [6]:
# Calc AMRF limits

sources['Ams'] = np.full(len(sources),np.nan)
sources['Atr'] = np.full(len(sources),np.nan)

## calculating for all sources

stage_min = 0  # pre-main sequence
stage_max = 5  # red giant branch
mass_min = 0  # [Msun]
mass_max = 8  # [Msun]

m2_vec = np.arange(0.1, 10, 0.1)

wavelength = gaia_passband['wl'].value  # [A]

Gflux1 = np.zeros(len(sources))
Gflux2 = np.zeros((len(sources), len(m2_vec)))
Ag1 = np.zeros(len(sources))
Ag2 = np.zeros((len(sources), len(m2_vec)))
q = np.zeros((len(sources), len(m2_vec)))
# PARSEC_path = '../data/PARSEC v1.2S/Gaia_lin/'
PARSEC_path = '../OCFit/gaiaDR2/grids/'
models = stam.getmodels.read_parsec(path=PARSEC_path)
for i in tqdm(range(len(sources))):
    idx = sources['idx'][i]
    if sources[i]['nss_solution_type'] not in ['Orbital','AstroSpectroSB1']:
        continue
    mh = sources['[Fe/H]'][i]
    age = sources['age'][i]
    ebv = sources['Av'][i]/3.1
    m1 = sources['m1'][i]
    
    try:
        track_idx = get_track_idx(mh,age)[-1]
        tracks = models[track_idx].copy()
        tracks.sort('Mini')
        idx = np.argmin(np.abs(tracks['Mass'] - m1))

        teff1 = 10**tracks['logTe'][idx]  # [K]
        logg1 = tracks['logg'][idx]
        r1 = mlogg2radius(m1*u.Msun, logg1)  # [Rsun]
        flux1 = blackbody(teff1, wavelength)*4*np.pi*r1**2
        Gflux1[i] = calc_synth_phot(wavelength, flux1, gaia_passband['gPb'].value).value
        Ag1[i],_ = extinction.get_AG_EBPRP(3.1*ebv,0,teff1)
        q[i, :] = m2_vec/m1  # mass ratio

        for j in range(len(m2_vec)):
            m2 = m2_vec[j]
            idx = np.argmin(np.abs(tracks['Mass'] - m2))
            teff2 = 10**tracks['logTe'][idx]  # [K]
            logg2 = tracks['logg'][idx]
            r2 = mlogg2radius(m2*u.Msun, logg2)  # [Rsun]
            flux2 = blackbody(teff2, wavelength, ebv=ebv)*4*np.pi*r2**2
            Gflux2[i,j] = calc_synth_phot(wavelength, flux2, gaia_passband['gPb'].value).value
            Ag2[i,j],_ = extinction.get_AG_EBPRP(3.1*ebv,0,teff2)

        Sms = Gflux2[i,:]/Gflux1[i]*10**(-0.4*(Ag2[i,:]-Ag1[i]))
        Ams = amrf(q[i, :], Sms)
        valid_idx = Sms < 1
        Ams = np.max(Ams[valid_idx]) 
        sources['Ams'][i] = Ams
        Str = 2*Gflux2[i,:]/Gflux1[i]
        Atr = amrf(2*q[i, :], Str)
        valid_idx = Str < 1
        Atr = np.max(Atr[valid_idx])
        sources['Atr'][i] = Atr
    except:
        print(f'i = {i}')

100%|██████████| 284/284 [17:53<00:00,  3.78s/it]


In [7]:
# Calc AMRF and class probabilities
new_sources = add_astrometric_parameters(sources)
new_sources['m2'] = new_sources['m1'] * new_sources['AMRF_q']
new_sources['m2_err'] = ((new_sources['m1_err'] * new_sources['AMRF_q'])**2 + (new_sources['m1'] * new_sources['AMRF_q_error'])**2)**(1/2)

100%|██████████| 284/284 [00:24<00:00, 11.47it/s]


In [12]:
sources = new_sources
cut1 = sources['significance'] > 158 * sources['period'].data**(-0.5)
cut2 = (sources['parallax_over_error'] > 20,000 *sources['period']**(-1))[0]
cut3 = sources['eccentricity_error'] < 0.079 * np.log(sources['period'].data) - 0.244

cut = cut1 & cut2 & cut3

sources = sources[cut]

# sources.write('../table_B.fits',overwrite=True)

In [18]:
cut1 = sources['m1'] / sources['m1_err'] > 50
cut2 = sources['classI_prob'] < 0.1
cut3 = sources['m2'] <= 1.4

cut = cut1 & cut2 & cut3

# sources[cut].write('../table_C.fits',overwrite=True)

In [21]:
import shutil
tbl = Table.read('../table_C.fits')

for i in range(len(tbl)):
    idx = tbl['idx'][i]
    cluster = tbl['cluster'][i]
    # cmd_fig_name = f'../img/cmd/{idx}_{cluster}.png'
    # cmd_new_name = f'../img/cmd_tableC/{idx}_{cluster}.png'
    # shutil.move(cmd_fig_name, cmd_new_name)
    interp_fig_name = f'../img/mass_interp/{idx}_{cluster}.png'
    interp_new_name = f'../img/mass_interpC/{idx}_{cluster}.png'
    shutil.move(interp_fig_name, interp_new_name)


# Cooling time and progenitor mass

In [3]:
## t_life vs m from evolutionary tracks/M_i from t_cluster, t_cool
from os import walk,path

def get_zarr():
    basedir = path.join('..','data','PARSEC v1.2S','evol_tracks')
    filenames = walk(basedir).__next__()[2]
    z_arr = []
    for n in filenames:
        z = n.split('Y')[0]
        z_arr.append(float(z[1:]))
    return np.unique(z_arr)

def get_mdict(mh):
    basedir = path.join('..','data','PARSEC v1.2S','evol_tracks')
    filenames = walk(basedir).__next__()[2]
    z_arr = get_zarr()
    z = 0.01524 * 10**mh
    z_nearest = z_arr[np.argmin(np.abs(z_arr - z))]
    z_str = 'Z'+f'{z_nearest}'

    m_arr = []
    s_arr = []
    for n in filenames:
        if z_str in n:
            m = n.split('M')[1].removesuffix('.DAT')
            if m.endswith('.HB'):
                m = m.removesuffix('.HB')
            m_arr.append(float(m))
            s_arr.append(m)
    tbl = Table({'mass':m_arr,'str':s_arr})
    tbl = tbl[tbl['mass'] <= 8]
    return tbl

def get_marr(mh):
    return np.unique(get_mdict(mh)['mass'])

def get_lifetime(mass,mh):
    ## lifetime for m>0.8
    basedir = path.join('..','data','PARSEC v1.2S','evol_tracks')
    filenames = walk(basedir).__next__()[2]
    z_arr = get_zarr()

    z = 0.01524 * 10**mh
    z_nearest = z_arr[np.argmin(np.abs(z_arr - z))]
    m_dict = get_mdict(mh)
    mass_str = m_dict[m_dict['mass'] == mass]['str']
    z_str = 'Z'+f'{z_nearest}'

    trackpath = ''
    for n in filenames:
        m1 = n.split('M')[1].removesuffix('.DAT')
        if m1.endswith('HB'):
            continue
        for m2 in mass_str:
            if m1 == m2 and z_str in n:
                trackpath = path.join(basedir,n)
                break

    if trackpath == '':
        print(f'No track found for '+m2+' '+z_str)

    trck = Table(np.genfromtxt(trackpath,names=True,dtype=None))
    return trck[trck['MODELL'] == 600 ]['AGE'][0]

def lifetime_vs_mass(mh):
    m_arr = get_marr(mh)
    m_arr = m_arr[m_arr > 0.8]
    t_arr = []
    for m in m_arr:
        t_arr.append(get_lifetime(m,mh))
    return m_arr,t_arr

def create_lifetime_vs_mass_tables():
    z_arr = get_zarr()
    mh_arr = np.log10(z_arr/0.01524)

    for mh in mh_arr:
        m_arr,t_arr = lifetime_vs_mass(mh)
        tbl = Table({'mass':m_arr,'lifetime':t_arr})
        tbl.write(path.join('..','data','MS_lifetime','parsec_V1.2S',f'lifetime_vs_mass{mh:.2f}.csv'),overwrite=True)

    return ''

def get_initial_mass(age,mh):
    ## age in Myr
    ## returns mass in solar mass
    basedir = path.join('..','data','MS_lifetime','parsec_V1.2S')
    z_arr = get_zarr()
    mh_arr = np.log10(z_arr/0.01524)
    mh = mh_arr[np.argmin(np.abs(mh_arr - mh))]
    filename = f'lifetime_vs_mass{mh:.2f}.csv'   
    df = pd.read_csv(path.join(basedir,filename))
    idx = np.argmin(np.abs(df['lifetime'] - age*1e6))
    return df['mass'][idx]

def get_wd_cooling_age(m2,t2):
    # in Myr
    model = WD_models.load_model(low_mass_model='Bedard2020',
                             middle_mass_model='Bedard2020',
                             high_mass_model='ONe',
                             atm_type='H',
                             HR_bands=('FUV-NUV', 'NUV'))

    m_logteff_to_agecool = WD_models.interp_xy_z_func(x=model['mass_array'],
                                                  y=model['logteff'],
                                                  z=model['age_cool'],
                                                  interp_type='linear')
    return m_logteff_to_agecool(m2, np.log10(t2)) * 1e3

# clstr_age = 762
# m2 = 0.81
# mh = 0.195
# omega = 0.0

# teff_lo = 12000
# teff_bst = 12500
# teff_hi = 13000

# wd_age_hi = get_wd_cooling_age(m2,teff_lo)
# wd_age_bst = get_wd_cooling_age(m2,teff_bst)
# wd_age_lo = get_wd_cooling_age(m2,teff_hi)
# ms_age_hi = clstr_age - wd_age_lo
# ms_age_bst = clstr_age - wd_age_bst
# ms_age_lo = clstr_age - wd_age_hi
# m1_hi = get_initial_mass(ms_age_lo,mh,omega)
# m1_bst = get_initial_mass(ms_age_bst,mh,omega)
# m1_lo = get_initial_mass(ms_age_hi,mh,omega)

# print(f'WD cooling age ~{int(wd_age_bst)} ({int(wd_age_lo)} to {int(wd_age_hi)})')
# print(f'Initial mass ~{m1_bst:.2f} ({m1_lo:.2f} to {m1_hi:.2f})')



In [ ]:
model = WD_models.load_model(low_mass_model='Bedard2020',
                             middle_mass_model='Bedard2020',
                             high_mass_model='ONe',
                             atm_type='H',
                             HR_bands=('V-NUV', 'NUV'))

m_teff_to_mag = WD_models.interp_xy_z_func(x=model['mass_array'],y=model['logteff'],z=model['Mag'],interp_type='linear')
m_teff_to_color = WD_models.interp_xy_z_func(x=model['mass_array'],y=model['logteff'],z=model['color'],interp_type='linear')

from astropy.coordinates import SkyCoord 
from astropy import units as u
for i in range(len(sources)):
    idx = sources[i]['idx'] 
    m2 = sources[i]['m2']
    t2 = sources[i]['teff2_min']
    ra = sources[i]['ra'] * u.deg
    dec = sources[i]['dec'] * u.deg
    parallax = sources[i]['parallax']
    logt2 = np.log10(t2)

    nuv = m_teff_to_mag(m2,logt2)
    fuv = m_teff_to_color(m2,logt2) + nuv

    nuv_apparent = nuv + 5 * np.log10(1000/parallax) - 5
    v_apparent = fuv + 5 * np.log10(1000/parallax) - 5
    if v_apparent < 19.5:
        print(f'idx {idx}, NUV = {nuv_apparent:.2f}, V = {v_apparent:.2f}, ' + SkyCoord(ra,dec).to_string('hmsdms'))